In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import logging
import json
import os

In [20]:
class ConfigData:
    WEB_DRIVER_FILE_LOCATION = ''
    listUrls = []
    Version = ""
    
    def __init__(self):
        self.GetConfigData()

    def GetConfigData(self):
        if os.path.isfile( "config.json" ):
            with open('config.json') as json_file:
                data = json.load(json_file)
            self.WEB_DRIVER_FILE_LOCATION = data['ChromeDriverLocation'] + 'chromedriver.exe'
            self.Version = data["Version"]
            self.listUrls = data['Targets']
        else:   #default
            self.WEB_DRIVER_FILE_LOCATION = 'C:/Users/myuser/path/to/chromedriver.exe'
            self.listUrls = ["https://www.idealista.com/venta-viviendas/madrid/hortaleza/apostol-santiago/",
                        "https://www.idealista.com/venta-viviendas/madrid/hortaleza/pinar-del-rey/"]

class DbPage:
    idList = []
    platformList = []
    titleList =  []
    priceList =  []
    parkingList = []
    habList = []
    nbanosList = []
    supList = []
    plantaList = []
    telefoneList =  []
    listHrefs = []
    listBarrio = []
    listZona = []
    listYear = []
    listMonth = []
    listDay = []
    listActive = []
    listYearInac = []
    listMonthInac = []
    listDayInac = []
    listCaract = []
    listComent = []
    
    
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.idList = []
        self.platformList = []
        self.titleList =  []
        self.priceList =  []
        self.parkingList = []
        self.habList = []
        self.nbanosList = []
        self.supList = []
        self.plantaList = []
        self.telefoneList =  []
        self.listHrefs = []
        self.listBarrio = []
        self.listZona = []
        self.listYear = []
        self.listMonth = []
        self.listDay = []
        self.listActive = []
        self.listYearInac = []
        self.listMonthInac = []
        self.listDayInac = []
        self.listCaract = []
        self.listComent = []

def setTitlePriceTelef(page, soup):
    page.titleList = soup.select("div.item-info-container > a.item-link")
    page.priceList = soup.select("div.item-info-container > div.price-row > span.item-price")
    
    for elem in soup.select("div.item-info-container > div.item-toolbar"):
        try:
            page.telefoneList.append( elem.select("span.icon-phone")[0].get_text() ) 
        except IndexError as error:
            page.telefoneList.append( "NA" )

def setIdealista(page, soup): #Encontrar id Idealista del piso
    date =  datetime.now()
    for elem in soup.select("div.item-info-container > a.item-link"):
        page.idList.append( int(elem["href"].split("/")[2]) )
        page.platformList.append("idealista")
        page.listYear.append( date.year )
        page.listMonth.append( date.month )
        page.listDay.append( date.day )
        
def setHabSupPl(page, soup):  #Encontrar Habitaciones, superficie y planta
    isHab = lambda a : True if "hab" in a else False
    isSup = lambda a : True if "m²" in a else False
    isPlanta = lambda a : True if ("planta".lower() in a.lower() or "Bajo".lower() in a.lower() or "exterior".lower() in a.lower() ) else False
    set_bit = lambda value, bit : value | (1<<bit)  #zero index

    for elem in soup.select("div.item-info-container"):
        auxStr = ""
        bitmap = 0  # hab,sup,planta as bits 000, 101, etc
        for subelem in elem.select("span.item-detail"):
            auxStr = subelem.get_text()
            if isHab(auxStr):
                page.habList.append( auxStr )
                bitmap = set_bit(bitmap, 2)
            if isSup(auxStr):
                page.supList.append( auxStr )
                bitmap = set_bit(bitmap, 1)
            if isPlanta(auxStr):
                page.plantaList.append( auxStr )
                bitmap = set_bit(bitmap, 0)
        if (bitmap & 0x1) == False:
            page.plantaList.append( "NA" )
        elif (bitmap & 0x2)>>1 == False:
            page.supList.append( "NA" )
        elif (bitmap & 0x4)>>2 == False:
            page.habList.append( "NA" )

def setParking(page, soup):  # Encontrar datos Garaje
    for elem in soup.select("div.item-info-container > div.price-row"):
        auxStr = ""
        parkingElem = elem.select("span.item-parking")
        if( not parkingElem):
            auxStr += "No Garaje "
        else:
            auxStr += parkingElem[0].get_text()
        page.parkingList.append( auxStr )

def setUrl(page, soup):  #Encontrar URL
    for elem in soup.select("div.item-info-container > a.item-link"):
        page.listHrefs.append( "https://www.idealista.com" + elem['href'] )
        
def insertPageInDf(page, df):
    for i in range(0,len(page.titleList)-1):
        row = [page.idList[i],
               page.platformList[i],
               page.titleList[i].get_text(),
               page.priceList[i].get_text(),
               page.habList[i],
               'NA',
               page.supList[i],
               page.plantaList[i],
               page.parkingList[i],
               page.telefoneList[i],
               page.listHrefs[i],
               page.listBarrio[i],
               page.listZona[i],
               page.listYear[i],
               page.listMonth[i],
               page.listDay[i],
               'Yes',
               'NA',
               'NA',
               'NA',
               'NA',
               'NA'
               ]
        if (df.IdPlat == int(page.idList[i])).any() :  #Evaluar tambien en caso de que si el precio es distinto, actualizar
            continue
        df.loc[len(df)] = row

def getNextPageUrl(soup, baseUrl): #compute next page URL
    out = ""
    try:
        out = mainUrl + soup.select("div.pagination > ul > li.next > a")[0]["href"]
    except IndexError as error:
        printAndLog("Last page in entry")
        out = "Last page in entry"
    return out

def setBarrioZona(Barrio, url, soup, page):
    Zona = url.split('/')[-2]
    for elem in soup.select("div.item-info-container > a.item-link"):
        page.listBarrio.append( Barrio )
        page.listZona.append( url.split('/')[-2].capitalize() )

def OpenBrowserAndFetchData(url, config):
    #Open webpage with selenium webdriver, extract page source, and close webdriver in order to avoid scaring target website 
    #with the multiple repeated requests that webdrive does.
    options = webdriver.ChromeOptions()
    options.page_load_strategy = 'eager'
    options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")
    driver = webdriver.Chrome(r"{}".format(config.WEB_DRIVER_FILE_LOCATION), options=options)
    driver.get(url)
    content = driver.page_source
    driver.quit()

    #Get Beautiful Soup Html object to work with
    soup = BeautifulSoup(content, 'html.parser')    
    return soup


def getDf():
    df = pd.DataFrame(columns=['IdPlat','Plat','Titulo','Precio','Hab','N_Banos','Sup','Planta','Garaje','Telefono','Url','Barrio','Zona','Year','Month', 'Day', 'Active', 'YearInact','MonthInact', 'DayInact', 'Caract', 'Comentario'])
    df = df.astype({"IdPlat": int})
    return df

def WorkZona(df, listUrls_entry, mainUrl, config):
    maxLoopCounts = 20
    counter = 0

    url = listUrls_entry
    Barrio = listUrls_entry.split('/')[-3].replace('-', ' ').title()
    
    while url != 'Last page in entry':
        if counter >= maxLoopCounts:
            break

        page = DbPage()

        printAndLog("Working.. url: " + url + ", counter: " + str(counter))
        soup = OpenBrowserAndFetchData(url, config)

        setTitlePriceTelef(page, soup)
        setIdealista(page, soup)
        print(page.idList)
        setHabSupPl(page, soup)
        setParking(page, soup)
        setUrl(page, soup)
        setBarrioZona(Barrio, url, soup, page)
        insertPageInDf(page, df)

        print(df[['IdPlat', 'Titulo']])
        url = getNextPageUrl(soup, mainUrl)
        
        printAndLog("Next Url: " + url)
        if url == 'Last page in entry':
            break
        time.sleep(NEXT_PAGE_DELAY) # NEXT_PAGE_DELAY seconds delay after request
        counter += 1
        
def BuildOutputDb(df_Db_in, df_today):
    df_Db_out = pd.concat([df_Db_in,df_today]).drop_duplicates(subset = ['IdPlat'], keep='first')
    df_Db_out = df_Db_out.reset_index(drop=True)
    return df_Db_out

#-------------------------------------------------------------------
#Spetial functions to find new and inactive entrys

def FindNewDf(df_Db_in, df_today):
    return df_today[df_today.IdPlat.isin(df_Db_in.IdPlat)==False]

def FindInactiveDf(df_Db_in, df_today):
    return df_Db_in[df_Db_in.IdPlat.isin(df_today.IdPlat)==False]

def MarkInactiveDf(df_InactConf, df_Db_out):
    index = df_Db_out[df_Db_out['IdPlat'].isin(df_InactConf.IdPlat)].index
    date =  datetime.now()
    
    for elem in index:
        InsertInCell(df_Db_out, elem, 'Active', "No")
        InsertInCell(df_Db_out, elem, 'YearInact', date.year)
        InsertInCell(df_Db_out, elem, 'MonthInact', date.month)
        InsertInCell(df_Db_out, elem, 'DayInact', date.day)
    df_InactConf.Active = 'No'
    df_InactConf.YearInact = date.year
    df_InactConf.MonthInact = date.month
    df_InactConf.DayInact = date.day
        
def getInactConf(df_Inactive, config):
    list = []
    printAndLog("Evaluating Inactive List: ")
    for i, elem in enumerate(df_Inactive.Url):
        print(elem)
        logging.info(elem)
        soup = OpenBrowserAndFetchData(elem, config)
        try:
            inactiveString = soup.select("div.deactivated-detail_container > h1")[0].get_text().strip()
            if inactiveString == 'Lo sentimos, este anuncio ya no está publicado':
                list.append(i)
                printAndLog("True Positive")
        except IndexError as error:
            printAndLog("False Positive")
        print("-------")
        logging.info("--------")
        time.sleep(15)

    return df_Inactive.iloc[ list ].reset_index(drop=True)

def InsertInCell(df_Db_out, row, Col, Value):
    df_Db_out.loc[df_Db_out.index[row], Col] = Value

def printAndLog(string):
    print(string)
    logging.info(string)

In [11]:
mainUrl = "https://www.idealista.com"

NEXT_PAGE_DELAY = 20
NEXT_ZONE_DELAY = 120

config = ConfigData()

logging.basicConfig(filename='RealEstateInfo.log', level=logging.INFO, format='%(levelname)s-%(asctime)s: %(message)s')
logging.basicConfig(encoding='utf-8')
logging.basicConfig(filename='RealEstateDebug.log', encoding='utf-8', level=logging.DEBUG, format='%(levelname)s-%(asctime)s: %(message)s')
logging.basicConfig(encoding='utf-8')

logging.info('Opening new log session')

config.listUrls

['https://www.idealista.com/venta-viviendas/madrid/hortaleza/apostol-santiago/',
 'https://www.idealista.com/venta-viviendas/madrid/hortaleza/pinar-del-rey/',
 'https://www.idealista.com/venta-viviendas/madrid/hortaleza/canillas/',
 'https://www.idealista.com/venta-viviendas/madrid/hortaleza/conde-orgaz-piovera/',
 'https://www.idealista.com/venta-viviendas/madrid/hortaleza/palomas/',
 'https://www.idealista.com/venta-viviendas/madrid/hortaleza/sanchinarro/',
 'https://www.idealista.com/venta-viviendas/madrid/hortaleza/valdebebas-valdefuentes/',
 'https://www.idealista.com/venta-viviendas/madrid/hortaleza/virgen-del-cortijo-manoteras/',
 'https://www.idealista.com/venta-viviendas/madrid/ciudad-lineal/costillares/']

In [12]:
#load pandas db
df_Db_in = getDf()
try:
    df_Db_in = pd.read_csv('dbPisos.csv', encoding='cp1252', index_col=False, keep_default_na=False)
except FileNotFoundError:
    printAndLog("First do one round of botting, to generate initial dbPisos.csv")

df_Db_in = df_Db_in.astype({"IdPlat": int})

df_Db_in

,IdPlat,Plat,Titulo,Precio,Hab,N_Banos,Sup,Planta,Garaje,Telefono,...,Zona,Year,Month,Day,Active,YearInact,MonthInact,DayInact,Caract,Comentario
0,92089313,idealista,"Piso en calle del Golfo de Salonica, Apóstol S...",498.000€,4 hab.,NA,153 m²,Bajo exterior con ascensor,Garaje incluido,918 006 403,...,Apostol-santiago,2021,1,6,Yes,NA,NA,NA,NA,NA
1,92108754,idealista,"Piso en calle de San Pedro de Cardeña, Apóstol...",160.000€,3 hab.,NA,76 m²,4ª planta exterior sin ascensor,No Garaje,914 879 865,...,Apostol-santiago,2021,1,6,Yes,NA,NA,NA,NA,NA
2,91949551,idealista,"Piso en Apóstol Santiago, Madrid",392.000€,4 hab.,NA,150 m²,5ª planta exterior con ascensor,No Garaje,910 754 756,...,Apostol-santiago,2021,1,6,Yes,NA,NA,NA,NA,NA
3,90218172,idealista,"Piso en Apóstol Santiago, Madrid",815.000€,6 hab.,NA,236 m²,2ª planta exterior con ascensor,Garaje incluido,916 356 954,...,Apostol-santiago,2021,1,6,Yes,NA,NA,NA,NA,NA
4,87295228,idealista,"Piso en calle de Añastro, Apóstol Santiago, Ma...",384.000€,2 hab.,NA,69 m²,4ª planta exterior con ascensor,No Garaje,915 357 200,...,Apostol-santiago,2021,1,6,Yes,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,92360443,idealista,"Piso en calle Arroyo del Charco del Pescador, ...",550.000€,4 hab.,NA,142 m²,4ª planta exterior con ascensor,Garaje incluido,914 892 496,...,Sanchinarro,2021,1,8,Yes,NA,NA,NA,NA,NA
1521,92318415,idealista,"Piso en calle de Américo Castro, 88, Sanchinar...",530.000€,3 hab.,NA,106 m²,2ª planta exterior con ascensor,Garaje incluido,616 934 449,...,Sanchinarro,2021,1,8,Yes,NA,NA,NA,NA,NA
1522,92340839,idealista,"Piso en calle Infanta Maria, 9, Sanchinarro, M...",235.000€,1 hab.,NA,49 m²,6ª planta exterior con ascensor,No Garaje,848 847 913,...,Sanchinarro,2021,1,8,Yes,NA,NA,NA,NA,NA
1523,87238595,idealista,"Piso en calle de la Dulzaina, 3, Costillares, ...",410.300€,3 hab.,NA,116 m²,1ª planta exterior con ascensor,No Garaje,911 797 153,...,Costillares,2021,1,8,Yes,NA,NA,NA,NA,NA


Main Bot Script

In [13]:
currentDateTime = lambda: datetime.now().strftime("%Y/%m/%d/ %H:%M:%S")

printAndLog( "Starting job: " + currentDateTime() )  
df_today = getDf()

for elem in config.listUrls:
    printAndLog( currentDateTime() + ", Current Url: " + elem  ) 
    WorkZona(df_today, elem, mainUrl, config)
    printAndLog( "Sleeping 2 min to be nice to servers: " + str(currentDateTime())   ) 
    time.sleep(NEXT_ZONE_DELAY)


Starting job: 2021/01/09/ 00:23:37
2021/01/09/ 00:23:37, Current Url: https://www.idealista.com/venta-viviendas/madrid/hortaleza/apostol-santiago/
Working.. url: https://www.idealista.com/venta-viviendas/madrid/hortaleza/apostol-santiago/, counter: 0
[92089313, 92108754, 91949551, 90218172, 87295228, 88574743, 88448484, 84992553, 90128541, 92268519, 92125508, 88827177, 92219308, 90987895, 88881308, 91959762, 91094571, 91148458, 92006700, 91752074, 88759681, 82298148, 27004286, 1833289, 92017339, 82917307, 1645789, 84834215, 89902044, 82830527]
      IdPlat                                             Titulo
0   92089313  Piso en calle del Golfo de Salonica, Apóstol S...
1   92108754  Piso en calle de San Pedro de Cardeña, Apóstol...
2   91949551                   Piso en Apóstol Santiago, Madrid
3   90218172                   Piso en Apóstol Santiago, Madrid
4   87295228  Piso en calle de Añastro, Apóstol Santiago, Ma...
5   88574743  Chalet pareado en calle de Alonso Saavedra, Ap...
6 

[39723924, 90218220, 92002539, 92002521, 91321167, 91192654, 91645483, 92035685, 92156000, 92128971, 92226864, 91515571, 91868999, 92366677, 92358174, 87857302, 90621376, 92157279, 92190586, 91470707, 90367063, 87835443, 87897053, 91868684, 91260650, 92045148, 91815795, 90724092, 90076360, 88083019]
       IdPlat                                             Titulo
0    92089313  Piso en calle del Golfo de Salonica, Apóstol S...
1    92108754  Piso en calle de San Pedro de Cardeña, Apóstol...
2    91949551                   Piso en Apóstol Santiago, Madrid
3    90218172                   Piso en Apóstol Santiago, Madrid
4    87295228  Piso en calle de Añastro, Apóstol Santiago, Ma...
..        ...                                                ...
101  91260650  Piso en calle del Mar de Aral, Pinar del Rey, ...
102  92045148  Piso en calle de Santa Virgilia, Pinar del Rey...
103  91815795  Piso en calle de los Arcos, 5, Pinar del Rey, ...
104  90724092         Piso en calle Arzúa, Pinar 

Working.. url: https://www.idealista.com/venta-viviendas/madrid/hortaleza/pinar-del-rey/pagina-8.htm, counter: 7
[90545567, 91488398, 90275458, 92351757, 91996295]
       IdPlat                                             Titulo
0    92089313  Piso en calle del Golfo de Salonica, Apóstol S...
1    92108754  Piso en calle de San Pedro de Cardeña, Apóstol...
2    91949551                   Piso en Apóstol Santiago, Madrid
3    90218172                   Piso en Apóstol Santiago, Madrid
4    87295228  Piso en calle de Añastro, Apóstol Santiago, Ma...
..        ...                                                ...
279  90559524  Piso en calle Santa Susana, 16, Pinar del Rey,...
280  90545567  Piso en Gran Vía de Hortaleza, 5, Pinar del Re...
281  91488398                      Piso en Pinar del Rey, Madrid
282  90275458  Piso en calle de la Costa del Sol, 6, Pinar de...
283  92351757                      Piso en Pinar del Rey, Madrid

[284 rows x 2 columns]
Last page in entry
Next Url: Las

Working.. url: https://www.idealista.com/venta-viviendas/madrid/hortaleza/canillas/pagina-7.htm, counter: 6
[91152654, 92032127, 91819988, 91887248, 91838170, 91686098, 83845661, 91885189, 91693371, 85894039, 90966731, 85779068, 1825157, 92260059, 89660259, 90117615, 91133982, 90597652, 90012308, 85309907, 91785313, 90610028, 87540763]
       IdPlat                                             Titulo
0    92089313  Piso en calle del Golfo de Salonica, Apóstol S...
1    92108754  Piso en calle de San Pedro de Cardeña, Apóstol...
2    91949551                   Piso en Apóstol Santiago, Madrid
3    90218172                   Piso en Apóstol Santiago, Madrid
4    87295228  Piso en calle de Añastro, Apóstol Santiago, Ma...
..        ...                                                ...
475  90597652  Piso en calle Motilla del Palancar, Canillas, ...
476  90012308   Piso en calle de Tribaldos, 64, Canillas, Madrid
477  85309907    Piso en calle de Silvano, 107, Canillas, Madrid
478  9178531

       IdPlat                                             Titulo
0    92089313  Piso en calle del Golfo de Salonica, Apóstol S...
1    92108754  Piso en calle de San Pedro de Cardeña, Apóstol...
2    91949551                   Piso en Apóstol Santiago, Madrid
3    90218172                   Piso en Apóstol Santiago, Madrid
4    87295228  Piso en calle de Añastro, Apóstol Santiago, Ma...
..        ...                                                ...
649  91433461  Dúplex en avenida de los Madroños, Conde Orgaz...
650  88494348              Dúplex en Conde Orgaz-Piovera, Madrid
651  87358652      Chalet pareado en Conde Orgaz-Piovera, Madrid
652  87022019      Chalet pareado en Conde Orgaz-Piovera, Madrid
653  86729416  Casa o chalet independiente en Conde Orgaz-Pio...

[654 rows x 2 columns]
Next Url: https://www.idealista.com/venta-viviendas/madrid/hortaleza/conde-orgaz-piovera/pagina-7.htm
Working.. url: https://www.idealista.com/venta-viviendas/madrid/hortaleza/conde-orgaz-piovera/

Working.. url: https://www.idealista.com/venta-viviendas/madrid/hortaleza/palomas/pagina-2.htm, counter: 1
[35125459, 92259727, 92138531, 91649151, 91505676, 87225732, 87158558, 90587879, 91191896, 32932193, 91539340, 92123589, 87237091, 29233292, 88917698, 90809495, 85365003, 38509300, 88221156, 39600427, 84279712, 87492646, 92039763, 92151572, 89433957, 89806651, 87812273, 91863101, 92273041, 91870712]
       IdPlat                                             Titulo
0    92089313  Piso en calle del Golfo de Salonica, Apóstol S...
1    92108754  Piso en calle de San Pedro de Cardeña, Apóstol...
2    91949551                   Piso en Apóstol Santiago, Madrid
3    90218172                   Piso en Apóstol Santiago, Madrid
4    87295228  Piso en calle de Añastro, Apóstol Santiago, Ma...
..        ...                                                ...
849  89433957  Chalet adosado en calle Villamiel de Cáceres, ...
850  89806651  Chalet pareado en avenida de los Prunos, Palom...
851  87

Working.. url: https://www.idealista.com/venta-viviendas/madrid/hortaleza/sanchinarro/pagina-5.htm, counter: 4
[89654742, 89411055, 91460475, 91460491, 91460364, 91460493, 91460188, 91460360, 92292943, 92090439, 91999276, 92053086, 86617326, 91218516, 92053120, 91243101, 92053052, 90719896, 92331936, 92230784, 91939143, 91763954, 91637664, 90071881, 82981540, 91590541, 91575054, 92280164, 88194334, 91640001]
        IdPlat                                             Titulo
0     92089313  Piso en calle del Golfo de Salonica, Apóstol S...
1     92108754  Piso en calle de San Pedro de Cardeña, Apóstol...
2     91949551                   Piso en Apóstol Santiago, Madrid
3     90218172                   Piso en Apóstol Santiago, Madrid
4     87295228  Piso en calle de Añastro, Apóstol Santiago, Ma...
...        ...                                                ...
1032  82981540        Piso en Americo Castro, Sanchinarro, Madrid
1033  91590541  Piso en avenida del Ingeniero Emilio Herrera

        IdPlat                                             Titulo
0     92089313  Piso en calle del Golfo de Salonica, Apóstol S...
1     92108754  Piso en calle de San Pedro de Cardeña, Apóstol...
2     91949551                   Piso en Apóstol Santiago, Madrid
3     90218172                   Piso en Apóstol Santiago, Madrid
4     87295228  Piso en calle de Añastro, Apóstol Santiago, Ma...
...        ...                                                ...
1179  88451005  Piso en avenida Juan Antonio Samaranch, 141, V...
1180  92116347  Piso en avenida Manuel Fraga Iribarne, 43, Val...
1181  92116356  Piso en avenida Manuel Fraga Iribarne, 43, Val...
1182  92351162  Piso en calle Félix Candela, 47, Valdebebas - ...
1183  92351159  Piso en calle Félix Candela, 47, Valdebebas - ...

[1184 rows x 2 columns]
Next Url: https://www.idealista.com/venta-viviendas/madrid/hortaleza/valdebebas-valdefuentes/pagina-3.htm
Working.. url: https://www.idealista.com/venta-viviendas/madrid/hortaleza/val

Working.. url: https://www.idealista.com/venta-viviendas/madrid/ciudad-lineal/costillares/pagina-2.htm, counter: 1
[92053095, 91894391, 90044776, 90029493, 92094776, 91753780, 91898933, 90017044, 91232407, 87436817, 85807942, 92058112, 92053699, 91855871, 89815517, 88957313, 91450205, 84860161, 84451281, 91936334, 92110257, 88498667, 92069104, 40305333, 85475092, 91676260, 90949238, 90251790, 88992464, 1814176]
        IdPlat                                             Titulo
0     92089313  Piso en calle del Golfo de Salonica, Apóstol S...
1     92108754  Piso en calle de San Pedro de Cardeña, Apóstol...
2     91949551                   Piso en Apóstol Santiago, Madrid
3     90218172                   Piso en Apóstol Santiago, Madrid
4     87295228  Piso en calle de Añastro, Apóstol Santiago, Ma...
...        ...                                                ...
1350  85475092           Piso en calle Bausá, Costillares, Madrid
1351  91676260       Piso en calle Bausá, 10, Costillares

In [14]:
df_today

,IdPlat,Plat,Titulo,Precio,Hab,N_Banos,Sup,Planta,Garaje,Telefono,...,Zona,Year,Month,Day,Active,YearInact,MonthInact,DayInact,Caract,Comentario
0,92089313,idealista,"Piso en calle del Golfo de Salonica, Apóstol S...",498.000€,4 hab.,NA,153 m²,Bajo exterior con ascensor,Garaje incluido,918 006 403,...,Apostol-santiago,2021,1,9,Yes,NA,NA,NA,NA,NA
1,92108754,idealista,"Piso en calle de San Pedro de Cardeña, Apóstol...",160.000€,3 hab.,NA,76 m²,4ª planta exterior sin ascensor,No Garaje,916 354 325,...,Apostol-santiago,2021,1,9,Yes,NA,NA,NA,NA,NA
2,91949551,idealista,"Piso en Apóstol Santiago, Madrid",392.000€,4 hab.,NA,150 m²,5ª planta exterior con ascensor,No Garaje,910 754 756,...,Apostol-santiago,2021,1,9,Yes,NA,NA,NA,NA,NA
3,90218172,idealista,"Piso en Apóstol Santiago, Madrid",815.000€,6 hab.,NA,236 m²,2ª planta exterior con ascensor,Garaje incluido,916 356 954,...,Apostol-santiago,2021,1,9,Yes,NA,NA,NA,NA,NA
4,87295228,idealista,"Piso en calle de Añastro, Apóstol Santiago, Ma...",384.000€,2 hab.,NA,69 m²,4ª planta exterior con ascensor,No Garaje,915 357 200,...,Apostol-santiago,2021,1,9,Yes,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,92211017,idealista,"Piso en Costillares, Madrid",235.000€,3 hab.,NA,76 m²,10ª planta exterior con ascensor,No Garaje,677 838 888,...,Costillares,2021,1,9,Yes,NA,NA,NA,NA,NA
1450,92081221,idealista,"Piso en Costillares, Madrid",540.000€,3 hab.,NA,126 m²,1ª planta exterior con ascensor,Garaje incluido,670 501 879,...,Costillares,2021,1,9,Yes,NA,NA,NA,NA,NA
1451,90670632,idealista,"Piso en arturo soria, 339, Costillares, Madrid",640.000€,4 hab.,NA,153 m²,4ª planta exterior con ascensor,Garaje incluido,667 488 622,...,Costillares,2021,1,9,Yes,NA,NA,NA,NA,NA
1452,90017438,idealista,"Piso en Costillares, Madrid",172.000€,3 hab.,NA,78 m²,Bajo exterior con ascensor,No Garaje,914 840 150,...,Costillares,2021,1,9,Yes,NA,NA,NA,NA,NA


In [15]:
df_Db_out = BuildOutputDb(df_Db_in, df_today)
df_Db_out

,IdPlat,Plat,Titulo,Precio,Hab,N_Banos,Sup,Planta,Garaje,Telefono,...,Zona,Year,Month,Day,Active,YearInact,MonthInact,DayInact,Caract,Comentario
0,92089313,idealista,"Piso en calle del Golfo de Salonica, Apóstol S...",498.000€,4 hab.,NA,153 m²,Bajo exterior con ascensor,Garaje incluido,918 006 403,...,Apostol-santiago,2021,1,6,Yes,NA,NA,NA,NA,NA
1,92108754,idealista,"Piso en calle de San Pedro de Cardeña, Apóstol...",160.000€,3 hab.,NA,76 m²,4ª planta exterior sin ascensor,No Garaje,914 879 865,...,Apostol-santiago,2021,1,6,Yes,NA,NA,NA,NA,NA
2,91949551,idealista,"Piso en Apóstol Santiago, Madrid",392.000€,4 hab.,NA,150 m²,5ª planta exterior con ascensor,No Garaje,910 754 756,...,Apostol-santiago,2021,1,6,Yes,NA,NA,NA,NA,NA
3,90218172,idealista,"Piso en Apóstol Santiago, Madrid",815.000€,6 hab.,NA,236 m²,2ª planta exterior con ascensor,Garaje incluido,916 356 954,...,Apostol-santiago,2021,1,6,Yes,NA,NA,NA,NA,NA
4,87295228,idealista,"Piso en calle de Añastro, Apóstol Santiago, Ma...",384.000€,2 hab.,NA,69 m²,4ª planta exterior con ascensor,No Garaje,915 357 200,...,Apostol-santiago,2021,1,6,Yes,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,92367168,idealista,"Piso en calle Agustín Calvo, Canillas, Madrid",198.000€,1 hab.,NA,50 m²,2ª planta exterior con ascensor,No Garaje,912 179 034,...,Canillas,2021,1,9,Yes,NA,NA,NA,NA,NA
1530,92364670,idealista,"Piso en calle de Ulises, 83, Canillas, Madrid",387.900€,2 hab.,NA,105 m²,1ª planta exterior con ascensor,Garaje incluido,912 186 067,...,Canillas,2021,1,9,Yes,NA,NA,NA,NA,NA
1531,91152654,idealista,"Piso en calle de Ulises, Canillas, Madrid",387.900€,2 hab.,NA,105 m²,1ª planta sin ascensor,Garaje incluido,910 312 463,...,Canillas,2021,1,9,Yes,NA,NA,NA,NA,NA
1532,32932193,idealista,"Chalet adosado en Palomas, Madrid",785.000€,4 hab.,NA,246 m²,NA,No Garaje,638 047 691,...,Palomas,2021,1,9,Yes,NA,NA,NA,NA,NA


In [18]:
#Write pandas df to db
df_Db_out.to_csv('dbPisos2.csv', encoding='cp1252',index=False)

In [21]:
#Get new and inactive entries
df_new = FindNewDf(df_Db_in, df_today).reset_index(drop=True)
df_Inactive = FindInactiveDf(df_Db_in, df_today).reset_index(drop=True)
df_InactConf = getInactConf(df_Inactive, config)
MarkInactiveDf(df_InactConf, df_Db_out)


Evaluating Inactive List: 
https://www.idealista.com/inmueble/90930981/
True Positive
-------
https://www.idealista.com/inmueble/82830527/
False Positive
-------
https://www.idealista.com/inmueble/91183578/
True Positive
-------
https://www.idealista.com/inmueble/91910534/
False Positive
-------
https://www.idealista.com/inmueble/91730568/
True Positive
-------
https://www.idealista.com/inmueble/91651383/
False Positive
-------
https://www.idealista.com/inmueble/90685790/
False Positive
-------
https://www.idealista.com/inmueble/89741843/
True Positive
-------
https://www.idealista.com/inmueble/91386161/
False Positive
-------
https://www.idealista.com/inmueble/92300457/
True Positive
-------
https://www.idealista.com/inmueble/92131170/
False Positive
-------
https://www.idealista.com/inmueble/92222454/
False Positive
-------
https://www.idealista.com/inmueble/92139465/
True Positive
-------
https://www.idealista.com/inmueble/92203691/
True Positive
-------
https://www.idealista.com/in

In [22]:
#Write results
if not os.path.exists('history'):
    os.makedirs('history')

date =  datetime.now()

todaysDirName = str(date.year) + date.strftime("%b") +  date.strftime('%d')
todaysDirNameLocation = 'history'+ '/' + todaysDirName

if not os.path.exists( todaysDirNameLocation ):
    os.makedirs( todaysDirNameLocation )

df_Db_in.to_csv( todaysDirNameLocation + '/' + 'dbIn_Pisos.csv', encoding='cp1252',index=False)
df_today.to_csv( todaysDirNameLocation + '/' + 'dbToday_Pisos.csv', encoding='cp1252',index=False)
df_Db_out.to_csv( todaysDirNameLocation + '/' + 'dbOut_Pisos.csv', encoding='cp1252',index=False)
df_new.to_csv( todaysDirNameLocation + '/' + 'dbNew_Pisos.csv', encoding='cp1252',index=False)
df_Inactive.to_csv( todaysDirNameLocation + '/' + 'dbInactive_Pisos.csv', encoding='cp1252',index=False)
df_InactConf.to_csv( todaysDirNameLocation + '/' + 'dbInactiveConf_Pisos.csv', encoding='cp1252',index=False)


#Write out df to overwrite standard input file
df_Db_out.to_csv('dbPisos.csv', encoding='cp1252',index=False)

In [ ]:
df_Db_out